In [1]:
import pandas
import csv
import io
import cx_Oracle

Загружаем xlsx

In [2]:
DF = pandas.read_excel('C:\\Users\\ELENA\\Desktop\\Organizations.xlsx')
org = DF['Название']
DF['Телефон'] = ""
print(org)

0    Северо-Западный институт повышения квалификаци...
1                        ФКУ "НАЛОГ-СЕРВИС" ФНС РОССИИ
2                                ФКУ «ОДЕЗ ФНС РОССИИ»
3                                            арваыраоы
4                ФБЛПУ "САНАТОРИЙ "РАДУГА" ФНС РОССИИ"
5                     ФБДОУ "ДОЛ "БЕРЕЗКА" ФНС РОССИИ"
6                ФБЛПУ "САНАТОРИЙ "ЭЛЛАДА" ФНС РОССИИ"
7                 ФБЛПУ «ЛРЦ «Подмосковье» ФНС России»
Name: Название, dtype: object


Открываем csv и проверяем, есть ли совпадение по названию. Если да, то записываем номер телефона.

In [3]:
path = "C:\\Users\\ELENA\\Desktop\\data-full.csv"
i = 0
for o in org:
    with io.open(path, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        first = True
        for r in reader:
            if first:
                first = False
                continue
            name = r[1]
            if name == o:
                DF['Телефон'][i] = r[4]
                break

    if pandas.isna(DF['Телефон'][i]):
        print("No info for {}".format(o))
    i += 1

No info for арваыраоы


C:\Users\ELENA\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [4]:
print(DF['Телефон'])

0     7 (812) 492-20-58
1     7 (495) 913-09-65
2     7 (495) 606-71-94
3                   NaN
4     7 (8622) 90-72-02
5     7 (496) 31-721-00
6    7 (861) 33-3-35-82
7     7 (495) 577-95-11
Name: Телефон, dtype: object


Создаём связь с базой данных

In [6]:
cx_Oracle.init_oracle_client(lib_dir="C:\\oracle\\instantclient_19_8")

In [7]:
conn = cx_Oracle.connect(r'login', r'password', 'db202009151316_high')

In [8]:
cursor = conn.cursor()

Создаём таблицу

In [12]:
cursor.execute("CREATE TABLE ORGANIZATIONS(name VARCHAR2(255),phnumber VARCHAR2(40))")

Если есть значение телефона, то записываем в таблицу

In [13]:
for c in range(len(org)):
    if not pandas.isna(DF['Телефон'][c]):
        cursor.execute("INSERT INTO ORGANIZATIONS(name, phnumber) VALUES(:name, :phnumber)", [DF['Название'][c], DF['Телефон'][c]])

In [14]:
cursor.execute("SELECT * FROM ORGANIZATIONS")
result = cursor.fetchall()
print(result)

[('Северо-Западный институт повышения квалификации Федеральной налоговой службы', '7 (812) 492-20-58'), ('ФКУ "НАЛОГ-СЕРВИС" ФНС РОССИИ', '7 (495) 913-09-65'), ('ФКУ «ОДЕЗ ФНС РОССИИ»', '7 (495) 606-71-94'), ('ФБЛПУ "САНАТОРИЙ "РАДУГА" ФНС РОССИИ"', '7 (8622) 90-72-02'), ('ФБДОУ "ДОЛ "БЕРЕЗКА" ФНС РОССИИ"', '7 (496) 31-721-00'), ('ФБЛПУ "САНАТОРИЙ "ЭЛЛАДА" ФНС РОССИИ"', '7 (861) 33-3-35-82'), ('ФБЛПУ «ЛРЦ «Подмосковье» ФНС России»', '7 (495) 577-95-11')]


In [15]:
conn.close()

In [11]:
#cursor.execute("DROP TABLE ORGANIZATIONS")